In [ ]:
from scapy.all import sniff, IP, TCP, UDP
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

# Initialize packet features dataframe
packet_data = pd.DataFrame(columns=['packet_size', 'ip_src', 'ip_dst', 'protocol', 'is_anomaly'])

# Anomaly detection model
isolation_forest = IsolationForest(contamination=0.05, random_state=42)

def process_packet(packet):
    if IP in packet:
        packet_size = len(packet)
        ip_src = packet[IP].src
        ip_dst = packet[IP].dst
        protocol = packet[IP].proto
        packet_data.loc[len(packet_data)] = [packet_size, ip_src, ip_dst, protocol, 0]

    if len(packet_data) >= 10:  # Every 10 packets, analyze for anomalies
        X = packet_data[['packet_size']]
        packet_data['is_anomaly'] = isolation_forest.fit_predict(X)
        anomalies = packet_data[packet_data['is_anomaly'] == -1]
        if not anomalies.empty:
            print("Anomalous packets detected!")
            print(anomalies)
        packet_data.drop(packet_data.index, inplace=True)  # Reset data after checking

# Start sniffing packets
print("Starting packet sniffing...")
sniff(prn=process_packet, count=50)  # Sniff 50 packets, but could be infinite for real-time detection